In [1]:
# Task 1
import mysql.connector as connector
from mysql.connector import Error
import numpy as np
from dataclasses import dataclass

from utils import get_default_connection_config, get_default_database, print_environment_info

try:
    connection = connector.connect(**get_default_connection_config(get_default_database()))
    print_environment_info(connection)
    print('Creating a standard cursor')
    cursor = connection.cursor()
except Error as e:
    print(f'Error: {e.errno}: {e.msg}')

Connector version: 8.0.32
Server version: 8.0.32
Creating a standard cursor


In [2]:
@dataclass
class Config:
    NUM_COURSES = 10
    NUM_CUSTOMERS = 5
    NUM_MENUS = 4
    NUM_ORDERS = 20

In [3]:
print('Populating table MenuItems')

np.random.seed(42)
menu_items = []
for course_index in range(Config.NUM_COURSES):
    num_dishes = np.random.randint(low=2, high=5)
    unit_prices = np.random.randint(low=10, high=100, size=num_dishes) + np.random.random(size=num_dishes)
    for menuitem_id, unit_price in enumerate(unit_prices):
        menu_items.append({
            'course_name': f'Course {course_index}',
            'starter_name': f'Starter {menuitem_id} for Course {course_index}',
            'desert_name': f'Desert {menuitem_id} for Course {course_index}',
            'unit_price': unit_price
        })

cursor.executemany("""
INSERT INTO MenuItems (CourseName, StarterName, DesertName, UnitPrice)
VALUES
(%(course_name)s, %(starter_name)s, %(desert_name)s, %(unit_price)s
);""", menu_items)

cursor.execute("COMMIT")

Populating table MenuItems


In [4]:

print('Populating table Menu')
menus = []
for menu_index in range(Config.NUM_MENUS):
    menus.append({
        'menu_name': f'Menu {menu_index}',
        'cuisine': f'Cuisine for {menu_index}',
    })
cursor.executemany("""
INSERT INTO Menus (MenuName, Cuisine)
VALUES
(%(menu_name)s, %(cuisine)s
);""", menus)
cursor.execute("COMMIT")

Populating table Menu


In [5]:
print('Populating table MenusAndMenuItems')

np.random.seed(42)
menu_and_menu_items = []
visited_menu_item_ids = set()
for menu_id in range(1, Config.NUM_MENUS + 1):
    num_dishes = np.random.randint(low=1, high=4)
    visited_menu_menuitem_mappings = set()
    for _ in range(num_dishes):
        while True:
            menuitem_id = np.random.randint(low=1, high=Config.NUM_COURSES + 1)
            mapping = (menu_id, menuitem_id)
            if mapping not in visited_menu_menuitem_mappings:
                visited_menu_menuitem_mappings.add(mapping)
                visited_menu_item_ids.add(menuitem_id)
                break
    for menu_id, menuitem_id in visited_menu_menuitem_mappings:
        menu_and_menu_items.append({
            'menu_id': menu_id,
            'menu_item_id': menuitem_id,
        })

all_menu_item_ids = set(range(1, Config.NUM_COURSES + 1))
left_over_menu_items = all_menu_item_ids - visited_menu_item_ids
for menuitem_id in left_over_menu_items:
    menu_and_menu_items.append({
        'menu_id': np.random.randint(low=1, high=Config.NUM_MENUS + 1),
        'menu_item_id': menuitem_id,
    })
cursor.executemany("""
INSERT INTO MenusAndMenuItems (MenuID, MenuItemID)
VALUES
(%(menu_id)s, %(menu_item_id)s
);""", menu_and_menu_items)

cursor.execute("COMMIT")

Populating table MenusAndMenuItems


In [6]:
print('Populating table Customers')

np.random.seed(42)
customers = []
for customer_idx in range(Config.NUM_CUSTOMERS):
    customers.append({
        'fullname': f'Full Name {customer_idx}',
        'contact_number': f'1{"".join(map(str, np.random.randint(low=0, high=10, size=9)))}',
        'email': f'customer_{customer_idx}@domain.com'
    })

cursor.executemany("""
INSERT INTO Customers (FullName, ContactNumber, Email)
VALUES
(%(fullname)s, %(contact_number)s, %(email)s
);""", customers)

cursor.execute("COMMIT")

Populating table Customers


In [7]:
print('Populating table Orders')

np.random.seed(42)
orders = []
exact_ordered_items = np.random.randint(low=1, high=len(menu_and_menu_items) + 1, size=Config.NUM_ORDERS).tolist()
customer_ids = np.random.randint(low=1, high=Config.NUM_CUSTOMERS + 1, size=Config.NUM_ORDERS).tolist()
quantities = np.random.randint(low=1, high=5, size=Config.NUM_ORDERS).tolist()
total_costs = []
for menu_and_menuitem_id, customer_id, quantity in zip(exact_ordered_items, customer_ids, quantities):
    cursor.execute(f"""
    SELECT mi.UnitPrice FROM MenuItems mi INNER JOIN MenusAndMenuItems mmi
    ON mi.MenuItemID = mmi.MenuItemID
    WHERE mmi.MenusAndMenuItemsID = {menu_and_menuitem_id}
    """)
    cost_of_menuitem = cursor.fetchone()[0]
    total_costs.append(cost_of_menuitem * quantity)

orders = list(zip(exact_ordered_items, customer_ids, quantities, total_costs))
cursor.executemany("""
INSERT INTO Orders (MenusAndMenuItemsID, CustomerID, Quantity, TotalCost)
VALUES
(%s, %s, %s, %s
);""", orders)

cursor.execute("COMMIT")

Populating table Orders


In [8]:
assert cursor.close()
assert connection.close() is None